In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

data = pd.read_csv("cleaned_input/train_consolidated.csv")

In [3]:
data.head()
data = data.drop(["msno", "Unnamed: 0"], axis=1)
data.shape

(853753, 122)

In [4]:
#data = pd.DataFrame([[1,2,3,5,6,7,8,9],[1,3,9,7,5,4,5,7]], columns=['a','b','c','d','e','f','g','is_churn'])

In [5]:
model_data = data.sample(frac=.1)

In [6]:
X_train = model_data.drop('is_churn', axis=1);
#replaced infinite values with zero. MAKE SURE THIS IS the right thing to do 
from numpy import inf
X_train[X_train == -inf] = 0
X_train[X_train == inf] = 0
y_train = model_data['is_churn'];

In [7]:
X_train.replace([np.inf, -np.inf], 0)

,trans_count,date_min,date_max,date_count,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,...,payment_method_id_encoded32,payment_method_id_encoded33,payment_method_id_encoded34,payment_method_id_encoded35,is_auto_renew_encoded0,is_auto_renew_encoded1,is_auto_renew_encoded2,is_cancel_encoded0,is_cancel_encoded1,is_cancel_encoded2
73619,29,20150101.0,20170228.0,10.0,2280.0,1094.0,744.0,678.0,5821.0,8788.0,...,0,0,0,1,0,0,1,0,1,0
311766,6,20170301.0,20170331.0,1.0,291.0,80.0,33.0,35.0,2996.0,1876.0,...,0,0,0,0,0,1,0,0,1,0
266624,19,20170301.0,20170331.0,1.0,299.0,42.0,65.0,33.0,940.0,1154.0,...,1,0,0,0,0,1,0,0,1,0
437950,14,20170311.0,20170321.0,1.0,7.0,2.0,3.0,2.0,27.0,39.0,...,0,0,1,0,0,0,1,0,1,0
616973,25,20170301.0,20170328.0,1.0,33.0,10.0,3.0,3.0,139.0,178.0,...,0,0,1,0,0,0,1,0,1,0
483257,23,20170301.0,20170331.0,1.0,347.0,91.0,33.0,35.0,913.0,751.0,...,0,0,0,0,0,0,1,0,1,0
402339,8,20160810.0,20170228.0,7.0,58.0,8.0,12.0,8.0,486.0,464.0,...,0,0,0,1,0,0,1,0,1,0
40025,1,20160306.0,20170215.0,3.0,162.0,41.0,17.0,14.0,111.0,286.0,...,1,0,0,0,0,1,0,0,1,0
197252,21,20150101.0,20170212.0,10.0,519.0,158.0,106.0,126.0,2200.0,2729.0,...,0,1,0,0,0,0,1,0,1,0
319759,21,20170301.0,20170331.0,1.0,183.0,59.0,18.0,16.0,556.0,570.0,...,0,1,0,0,0,0,1,0,1,0


In [8]:
X_train.isnull().any()

trans_count                    False
date_min                       False
date_max                       False
date_count                     False
num_25_sum                     False
num_50_sum                     False
num_75_sum                     False
num_985_sum                    False
num_100_sum                    False
num_unq_sum                    False
total_secs_sum                 False
city                           False
bd                             False
registered_via                 False
registration_init_year         False
registration_init_month        False
registration_init_date         False
payment_method_id              False
payment_plan_days              False
plan_list_price                False
actual_amount_paid             False
is_auto_renew                  False
is_cancel                      False
discount                       False
is_discount                    False
amt_per_day                    False
membership_duration            False
m

In [9]:
print(np.all(np.isfinite(X_train)))
print(np.any(np.isnan(X_train)))

True
False


In [10]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train, scoring="neg_log_loss", train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [12]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_knn = {'n_neighbors':[1,3,6,10], 'weights':['uniform','distance']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}
#param_ann = {hidden_layer_sizes}
param_rfc = {'n_estimator':[5,10,15,20], 'max_features':[10,100,500,1000]}


In [ ]:
model_list = {LogisticRegression():param_logit, KNeighborsClassifier():param_knn,
             SVC():param_svc, RandomForestClassifier():param_rfc};
model_list = {LogisticRegression():param_logit}

In [ ]:
for model,param in model_list.items():
    gscv = GridSearchCV(model, param);
    print("learning curve for\t{}".format(model))
    plot_learning_curve(gscv, X_train, y_train)

learning curve for	LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
